# SB_CNN Example

In [13]:
%load_ext autoreload
%autoreload 2

import sys
import os
import glob
import numpy as np
import argparse

sys.path.append('../')
from dcase_models.utils.files import load_json, mkdir_if_not_exists
from dcase_models.data.data_generator import DataGenerator
from dcase_models.data.datasets import UrbanSound8k
from dcase_models.data.datasets import ESC10
from dcase_models.data.features import MelSpectrogram
from dcase_models.model.models import SB_CNN
from dcase_models.data.scaler import Scaler

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Define parameters

In [14]:
# features parameters
sequence_time = 2.0
sequence_hop_time = 0.5
audio_hop = 670
audio_win = 1024
n_fft = 1024
sr = 22050
mel_bands = 128

# normalizer
normalizer = 'minmax'

# train parameters
early_stopping = 10
epochs = 5
considered_improvement = 0
learning_rate = 0.001
batch_size = 64
verbose = 1
optimizer = 'Adam'

# dataset parameters
#dataset_path = '../../../../data/pzinemanas/UrbanSound8K'
dataset_path = '../../../../data/pzinemanas/ESC50'
audio_folder = 'audio22050'
feature_folder = 'features'

## Init Feature Extractor and Data Generator

In [16]:
# Init Feature Extractor
feature_extractor = MelSpectrogram(sequence_time=sequence_time, 
                                   sequence_hop_time=sequence_hop_time, 
                                   audio_win=audio_win, 
                                   audio_hop=audio_hop, 
                                   n_fft=n_fft, 
                                   sr=sr, mel_bands=mel_bands)

# Init Data Generator
dataset = ESC10(dataset_path)

label_list ['dog', 'rooster', 'rain', 'sea_waves', 'crackling_fire', 'crying_baby', 'sneezing', 'clock_tick', 'helicopter', 'chainsaw']


In [17]:
data_generator = DataGenerator(dataset, feature_extractor)

## Extract Features

In [18]:
data_generator.extract_features()
print('Done!')

Done!


## Load data

In [19]:
print('Loading data... ')
data_generator.load_data()
print('Done!')

Loading data... 
fold: [############################################################] 5/5
Done!


## Fit scaler

In [20]:
fold_test = 'fold1'

X_train, Y_train, X_val, Y_val = data_generator.get_data_for_training(fold_test)
scaler = Scaler(normalizer=normalizer)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

## Init Model

In [21]:
n_frames_cnn = X_train.shape[1]
n_freq_cnn = X_train.shape[2]
n_classes = Y_train.shape[1]
print(n_frames_cnn, n_freq_cnn, n_classes)

model_container = SB_CNN(model=None, model_path=None, n_classes=n_classes, 
                         n_frames_cnn=n_frames_cnn, n_freq_cnn=n_freq_cnn)

model_container.model.summary()

64 128 10
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 64, 128)           0         
_________________________________________________________________
lambda (Lambda)              (None, 64, 128, 1)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 60, 124, 24)       624       
_________________________________________________________________
maxpool1 (MaxPooling2D)      (None, 30, 62, 24)        0         
_________________________________________________________________
batchnorm1 (BatchNormalizati (None, 30, 62, 24)        96        
_________________________________________________________________
conv2 (Conv2D)               (None, 26, 58, 48)        28848     
_________________________________________________________________
maxpool2 (MaxPooling2D)      (None, 6, 29, 48)         0         


## Train model

In [22]:
exp_folder = './'

train_arguments = {'early_stopping': early_stopping,
                  'epochs': epochs,
                  'considered_improvement': considered_improvement,
                  'learning_rate': learning_rate,
                  'batch_size': batch_size,
                  'verbose': verbose,
                  'optimizer': optimizer}

model_container.train(X_train, Y_train, X_val, Y_val, weights_path=exp_folder, **train_arguments)

Epoch 1/5
2640/2640 [==============================] - 1s 366us/step - loss: 2.3939
Acc = 0.6125 - Best val Acc: 0.6125 (IMPROVEMENT, saving)

Epoch 2/5
2640/2640 [==============================] - 0s 160us/step - loss: 1.4687
Acc = 0.6125 - Best val Acc: 0.6125 (0)

Epoch 3/5
2640/2640 [==============================] - 0s 181us/step - loss: 1.1995
Acc = 0.8625 - Best val Acc: 0.8625 (IMPROVEMENT, saving)

Epoch 4/5
2640/2640 [==============================] - 0s 172us/step - loss: 0.9689
Acc = 0.7875 - Best val Acc: 0.8625 (2)

Epoch 5/5
2640/2640 [==============================] - 0s 179us/step - loss: 0.8988
Acc = 0.7875 - Best val Acc: 0.8625 (2)



## Evaluate Model

In [23]:
# Load best_weights
model_container.load_model_weights(exp_folder)

# Test model
X_test, Y_test = data_generator.get_data_for_testing(fold_test)
X_test = scaler.transform(X_test)
results = model_container.evaluate(X_test, Y_test)

print(results['accuracy'])

0.75
